In [ ]:
%load_ext autoreload
%autoreload 2

import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import lib

PROJECT_ROOT = os.path.abspath(os.path.join(os.curdir, os.pardir))
DATA_ROOT = os.path.join(PROJECT_ROOT, 'data')

PROJECT_ROOT

In [ ]:
config = {
    "objects": [
        {"src_bbox": [[338, 95], [907, 95], [910, 660], [330, 660]]},
    ]
}

VIDEO_FNAME = '2_5262567863796895591.mp4'
VIDEO_FPATH = os.path.join(DATA_ROOT, "AnalogDevicesImgs", VIDEO_FNAME)

In [ ]:
reader = lib.VideoReader(VIDEO_FPATH)

frame = reader[0]

roi_poly_pnts = config["objects"][0]["src_bbox"]

screen_extr = lib.IndicatorScreenExtractor(roi_pnts=roi_poly_pnts, ind_size=400)
screen_extr.fit(frame)

screen_frame = screen_extr.transform(frame)

arr_det = lib.ArrowDetector()

lines, arrow_angles = arr_det.transform(screen_frame)

screen_canvas = lib.drawLines(screen_frame, lines)

arrow_angles_deg = np.rad2deg(arrow_angles)

lib.normalize_angles_deg(arrow_angles_deg, copy=False)

units_convertor = lib.AngleUnitsConvertor()
units_convertor.fit(X=[-11, 19.5, -28.5], y=[11, 21, 5])

print(arrow_angles_deg)
print(np.mean(arrow_angles_deg))

_, ax = plt.subplots(ncols=3, figsize=(15, 9))
ax[0].imshow(frame)
ax[1].imshow(screen_frame)
ax[2].imshow(screen_canvas)


In [ ]:
import ipywidgets as widgets

def slider_handler(x):
    frame = reader[x]
    screen_frame = screen_extr.transform(frame)
    lines, arrow_angles = arr_det.transform(screen_frame)
    screen_canvas = lib.drawLines(screen_frame, lines)
    arrow_angles_deg = np.rad2deg(arrow_angles)
    lib.normalize_angles_deg(arrow_angles_deg, copy=False)

    text_writer = lib.ImageTextWriter()
    text_writer.fontColor = (0, 127, 0)
    text_writer.fontScale = 0.7

    mean_angle = np.mean(arrow_angles_deg)
    mean_unit = units_convertor.transform([mean_angle])

    print(mean_angle)
    print(mean_unit)

    text_writer.putText(screen_frame, f"{round(mean_angle, 2)} | {round(float(mean_unit[0]), 2)}", (10, 20))

    plt.imshow(screen_frame)
    plt.show()

    # -11 ~ 11
    # 19.5 ~ 21
    # -28.5 ~ 5

widgets.interactive(slider_handler, x=widgets.IntSlider(max=len(reader)))